In [ ]:
import sys
import os
module_path = os.path.abspath(os.path.join('../../../../src'))
if module_path not in sys.path:
    sys.path.append(module_path)

%config InlineBackend.figure_format = 'retina'

In [ ]:
from society.agents import AnalyticalAgent
from society.structure.network import ArtificialNetwork
from society.beliefs import Distribution, BeliefDistribution
from society.structure import AgentNetwork
from updates import Interactions, UpdateRule
from simulation import Simulation, Insights
from plotting.basic import *
from matplotlib import pyplot as plt
from computations.insights.relative import diversity, disagreement
from plotting.summaries import *
import pandas as pd

In [ ]:
linespace = Distribution(type="linespace", range=(-1,1))
update_rule = UpdateRule(evaluation_bias=0.3, rewire_probability=1)
interactions = Interactions(update_rule, interacting_agents=True)
insights = Insights()
simulation = Simulation(1000, insights)
network = ArtificialNetwork(100, "random_graph", p = 0.2)
unique = Distribution(type="unique", value=0.1)
belief_distribution = BeliefDistribution(unique, linespace)
agent_network = AgentNetwork(belief_distribution, network, agent_type=AnalyticalAgent)
update_rule = UpdateRule(assimilation_bias=0.05, evaluation_bias=0, rewire_probability=0)
interactions = Interactions(update_rule, interacting_agents=True)
agent_network.network =  ArtificialNetwork(100, "random_graph", p = 1)


In [ ]:
def compute_div_dis(assim, ev, rw, densities):
    unique = Distribution(type="unique", value=0.1)
    belief_distribution = BeliefDistribution(unique, linespace)
    agent_network = AgentNetwork(belief_distribution, network, agent_type=AnalyticalAgent)
    update_rule = UpdateRule(assimilation_bias=assim, evaluation_bias=ev, rewire_probability=rw)
    interactions = Interactions(update_rule, interacting_agents=True)

    repetitions = 100
    results_diversity = []
    results_disagreement= []
    for density in densities:
        for i in range(repetitions):
            agent_network.network =  ArtificialNetwork(100, "random_graph", p = density)
            simulation.run(interactions, agent_network)
            beliefs = simulation.insights.belief_evolution.T[simulation.iterations]
            mean = np.mean(beliefs)
            beliefs = simulation.insights.belief_evolution.T[simulation.iterations]
            edges = agent_network.get_edges()
            agents = agent_network.agent_lookup
            results_disagreement.append((disagreement(agents, edges), density))
            results_diversity.append((diversity(agent_network, mean), density))

    df_diversity = pd.DataFrame(results_diversity, columns=['Diversity', 'Density'])
    df_disagreement = pd.DataFrame(results_disagreement, columns=['Disagreement', 'Density'])
    return df_diversity, df_disagreement

In [ ]:
densities = [0.8, 0.6, 0.4, 0.2, 0.1, 0.08, 0.06, 0.04, 0.02, 0.01]
df1_div, df1_dis = compute_div_dis(assim=0, ev=0.3,rw=0, densities=densities)
df1_div.to_csv('../../../../results/data/random_graph/no_rewire/dis_evbias_0.1.csv')
df1_dis.to_csv('../../../../results/data/random_graph/no_rewire/dis_evbias_0.1.csv')